写一下L1、L2正则化公式，分别适用于什么场景，从数学角度讲一下两者的区别

# L1 与 L2 正则化

- **公式：**  
  - L1 正则化：  
    $$
    \mathcal{L}_{L1} = \mathcal{L} + \lambda \sum |w_i|
    $$
  - L2 正则化：  
    $$
    \mathcal{L}_{L2} = \mathcal{L} + \lambda \sum w_i^2
    $$

- **适用场景：**  
  - L1：适合高维稀疏场景，用于特征选择，会产生稀疏解。  
  - L2：适合大部分特征都有用的场景，用于防止过拟合，让模型更稳定。  

- **数学区别：**  
  - L1 的约束区域是菱形，带有尖角，解更容易落在坐标轴上 → 某些参数为 0 → 稀疏。  
  - L2 的约束区域是圆形/球体，光滑无尖点，解会均匀收缩但不为 0 → 平滑。  

---

📌 **一句话总结：**  
L1 产生稀疏解（特征选择），L2 产生平滑解（防过拟合，稳定性好）。

写一下逻辑回归的表达式，参数怎么推导

# 逻辑回归

## 1. 模型表达式

逻辑回归用于二分类问题，预测概率为：

$$
P(y=1 \mid \mathbf{x}) = \sigma(\mathbf{w}^\top \mathbf{x} + b)
= \frac{1}{1 + e^{-(\mathbf{w}^\top \mathbf{x} + b)}}
$$

其中：
- $\mathbf{x} \in \mathbb{R}^d$：特征向量  
- $\mathbf{w}$：参数向量  
- $b$：偏置  
- $\sigma(z)$：Sigmoid 函数  

同时：
$$
P(y=0 \mid \mathbf{x}) = 1 - P(y=1 \mid \mathbf{x})
$$

---

## 2. 参数估计（极大似然）

训练集 $\{(\mathbf{x}_i, y_i)\}_{i=1}^n$，似然函数为：

$$
L(\mathbf{w}, b) = \prod_{i=1}^n P(y_i \mid \mathbf{x}_i)
= \prod_{i=1}^n \sigma(\mathbf{w}^\top \mathbf{x}_i + b)^{y_i} \, \big(1-\sigma(\mathbf{w}^\top \mathbf{x}_i + b)\big)^{1-y_i}
$$

取对数得到对数似然函数：

$$
\ell(\mathbf{w}, b) = \sum_{i=1}^n \Big[ y_i \log \sigma(\mathbf{w}^\top \mathbf{x}_i + b) + (1-y_i)\log(1-\sigma(\mathbf{w}^\top \mathbf{x}_i + b)) \Big]
$$

---

## 3. 参数推导（梯度）

Sigmoid 的导数：
$$
\sigma'(z) = \sigma(z)(1-\sigma(z))
$$

对参数求偏导：

- 对 $\mathbf{w}$：
$$
\frac{\partial \ell}{\partial \mathbf{w}} 
= \sum_{i=1}^n \big(y_i - \sigma(\mathbf{w}^\top \mathbf{x}_i + b)\big)\mathbf{x}_i
$$

- 对 $b$：
$$
\frac{\partial \ell}{\partial b} 
= \sum_{i=1}^n \big(y_i - \sigma(\mathbf{w}^\top \mathbf{x}_i + b)\big)
$$

---

## 4. 参数解法

逻辑回归没有解析解，因为对数似然是非线性的。  
一般采用数值优化方法（梯度下降 / 牛顿法）：

$$
\mathbf{w} \leftarrow \mathbf{w} + \eta \sum_{i=1}^n (y_i - \hat{y}_i)\mathbf{x}_i
$$

其中 $\hat{y}_i = \sigma(\mathbf{w}^\top \mathbf{x}_i + b)$，$\eta$ 为学习率。

---

✅ **总结：**  
逻辑回归通过 Sigmoid 将线性模型转为概率输出，参数用极大似然估计，最后通过迭代优化求解。

# Dropout 在训练与测试阶段的区别及差异平衡

## 训练阶段
- **随机屏蔽神经元**：以概率 \(p\) 将一部分 hidden states 置为 0。  
- **缩放保留神经元**：为了保证期望不变，保留的神经元除以 \(1-p\)。  
公式如下：  
$h^{(train)} = \frac{m \odot h}{1-p}, \quad m \sim \text{Bernoulli}(1-p)$
其中：
- \(h\)：隐层输出（hidden states）
- \(m\)：随机 mask 向量
- \(p\)：dropout rate

这样可以看作训练时在不同的“子网络”上进行参数更新，达到正则化效果。

---

## 测试阶段
- **不再随机屏蔽**：推理需要稳定性，因此不再对神经元进行随机丢弃。  
- **直接使用全量神经元**：由于训练时做了缩放补偿，测试时可直接使用完整的 \(h\)，分布保持一致。  
公式：
$h^{(test)} = h$

---

## 如何平衡训练和测试的差异
1. **缩放补偿**：训练阶段对保留单元除以 \(1-p\)，保证期望一致。  
2. **蒙特卡洛 Dropout**：推理时继续使用 dropout，多次采样前向传播，用于不确定性估计。  
3. **合理选择 dropout rate**：一般在 0.1–0.5 之间，过大会导致 gap 过大。  
4. **与 BatchNorm 搭配注意**：通常先做 BN，再做 Dropout，以避免统计量偏差。

---

## 面试简洁回答
> Dropout 在训练阶段会随机屏蔽神经元并缩放保留单元；测试阶段则使用全量神经元不再屏蔽。通过缩放补偿，使训练和测试的分布保持一致，从而平衡差异。